In [1]:
! pip install transformers datasets py7zr evaluate rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5

In [2]:
from random import randrange

from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Dataset

We are going to fine-tune the [Flan-T5](https://huggingface.co/google/flan-t5-small) model to summarize dialogue/ chat threads similar to what is done in the ChatGPT interface. We will use the [SAMSum](https://huggingface.co/datasets/samsum) dataset. The SAMSum dataset contains about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

(Thanks to [Philipp Schmid](https://www.philschmid.de/) for great examples of this task that were adapted for this notebook)

In [3]:
dataset = load_dataset("samsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 14732
Test dataset size: 819


In [4]:
dataset['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [13]:
#pick a random sample to evaluate the model later
sample = dataset['test'][randrange(len(dataset["test"]))]
sample

{'id': '13729697',
 'dialogue': "Sophie: When r u going to Poanań?\r\nMurphy: On Tuesday. \r\nSophie: And you're coming back the same day?\r\nMurphy: Yes, in the afternoon, but I don't know the exact hour. ",
 'summary': 'Murphy is going to Poanań on Tuesday and coming back on the same day in the afternoon.'}

# Pre-process data

In [5]:
# We will use the Flan T5 tokenizer to help us pre-process the data.
model_id="google/flan-t5-small"

# Load tokenizer of Flan-T5
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([
    dataset["train"], 
    dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), 
    batched=True, 
    remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([
    dataset["train"], 
    dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), 
    batched=True, 
    remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/15551 [00:00<?, ? examples/s]

Max target length: 95


In [7]:
def preprocess_function(sample,padding="max_length"):

    # Add prefix to the input prompt for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, 
                             padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], 
                       max_length=max_target_length, padding=padding, 
                       truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels 
    # by -100 when we want to ignore padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, 
                                remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


# Base/ Foundation Model

In [11]:
# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

Before we fine-tune, let's see how the model performs on our sample!

In [14]:
base_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# print dialogue and reference summary
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"reference summary: \n{sample['summary']}\n---------------")

# print model summary
base_result = base_summarizer(sample["dialogue"])
print(f"base flan-t5-base summary:\n{base_result[0]['summary_text']}")

Your max_length is set to 200, but you input_length is only 53. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


dialogue: 
Sophie: When r u going to Poanań?
Murphy: On Tuesday. 
Sophie: And you're coming back the same day?
Murphy: Yes, in the afternoon, but I don't know the exact hour. 
---------------
reference summary: 
Murphy is going to Poanań on Tuesday and coming back on the same day in the afternoon.
---------------


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


base flan-t5-base summary:
Sophie is going to Poana on Tuesday. Murphy is coming back the same day, but she doesn't know the exact hour.


# Fine-tune

In [15]:
# We will use the Rouge metric for evaluations. ROUGE, or Recall-Oriented 
# Understudy for Gisting Evaluation, is a set of metrics and a software package 
# used for evaluating automatic summarization and machine translation software 
# in natural language processing. The metrics compare an automatically produced 
# summary or translation against a reference or a set of references 
# (human-produced) summary or translation.
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

In [16]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [17]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-samsum",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"flan-t5-samsum/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

In [18]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.810700,1.663632,44.254200,20.331600,34.880300,41.038200,36.133089
2,1.746400,1.640039,44.497900,20.638900,35.084300,41.215500,36.081807
3,1.689400,1.638276,44.682000,20.755600,35.154300,41.269900,36.825397
4,1.667300,1.630642,44.869400,21.005400,35.460600,41.429500,36.702076
5,1.637100,1.630473,44.776000,20.659400,35.336900,41.266900,36.688645


TrainOutput(global_step=9210, training_loss=1.7131637656080347, metrics={'train_runtime': 1938.2937, 'train_samples_per_second': 38.002, 'train_steps_per_second': 4.752, 'total_flos': 1.369269457649664e+16, 'train_loss': 1.7131637656080347, 'epoch': 5.0})

 # Evaluate

In [19]:
# Evaluate using the same trainer.
trainer.evaluate()

{'eval_loss': 1.6304731369018555,
 'eval_rouge1': 44.776,
 'eval_rouge2': 20.6594,
 'eval_rougeL': 35.3369,
 'eval_rougeLsum': 41.2669,
 'eval_gen_len': 36.688644688644686,
 'eval_runtime': 121.0938,
 'eval_samples_per_second': 6.763,
 'eval_steps_per_second': 0.851,
 'epoch': 5.0}

# Try it out!

In [24]:
# load model and tokenizer from with pipeline
finetuned_summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# select a random test sample
# sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"reference summary: \n{sample['summary']}\n---------------")
print(f"base model summary: \n{base_result[0]['summary_text']}\n---------------")

# summarize dialogue
finetuned_result = finetuned_summarizer(sample["dialogue"])

print(f"finetuned model summary:\n{finetuned_result[0]['summary_text']}")

Your max_length is set to 200, but you input_length is only 53. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=26)


dialogue: 
Sophie: When r u going to Poanań?
Murphy: On Tuesday. 
Sophie: And you're coming back the same day?
Murphy: Yes, in the afternoon, but I don't know the exact hour. 
---------------
reference summary: 
Murphy is going to Poanań on Tuesday and coming back on the same day in the afternoon.
---------------
base model summary: 
Sophie is going to Poana on Tuesday. Murphy is coming back the same day, but she doesn't know the exact hour.
---------------
finetuned model summary:
Murphy is going to Poana on Tuesday. He's coming back the same day in the afternoon, but he doesn't know the exact hour.
